# LSIMLを実際のマーケットデータに当てはめて計算してみる

## 準備

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
import calc_LSIML

In [3]:
csv1 = pd.read_csv(
    r"C:\Users\ara-d\stock_analisis\mitsuifin_1sec_2018-09-03_2019-08-30.csv"
)
csv2 = pd.read_csv(
    r"C:\Users\ara-d\stock_analisis\mitsuifin_1sec_2019-09-02_2021-07-30.csv"
)

In [4]:
sec1 = pd.concat([csv1, csv2])
# 対数価格を列に加える
sec1["log_price"] = np.log(sec1["price"])

In [5]:
# データが存在するような日付のリスト
Days_list = list(dict.fromkeys(sec1["date"]))

## C_c, P_cを予め計算する

In [6]:
overlap_rate = 0
b_list = [10, 50,100]
alpha = 0.4

nの一覧を取得する

In [7]:
n_array = np.array(list(set(sec1.groupby("date")["log_price"].count())))
n_array = n_array - np.array([1] * len(n_array))
n_array

array([17930, 17293, 17819, 17948, 17956, 17957, 17830, 17834, 17963,
       17837, 17839, 17978, 17988, 17996, 17999, 17873, 17886, 17639,
       17770, 17898, 17645, 17646])

In [8]:
c_list_dict = {}
for n in n_array:
    c_list = []
    for b in b_list:
        c_list.append(calc_LSIML.calc_c(n, b, overlap_rate))
    c_list_dict[n] = c_list

c_list_dict

{17930: [1793, 358, 179],
 17293: [1729, 345, 172],
 17819: [1781, 356, 178],
 17948: [1794, 358, 179],
 17956: [1795, 359, 179],
 17957: [1795, 359, 179],
 17830: [1783, 356, 178],
 17834: [1783, 356, 178],
 17963: [1796, 359, 179],
 17837: [1783, 356, 178],
 17839: [1783, 356, 178],
 17978: [1797, 359, 179],
 17988: [1798, 359, 179],
 17996: [1799, 359, 179],
 17999: [1799, 359, 179],
 17873: [1787, 357, 178],
 17886: [1788, 357, 178],
 17639: [1763, 352, 176],
 17770: [1777, 355, 177],
 17898: [1789, 357, 178],
 17645: [1764, 352, 176],
 17646: [1764, 352, 176]}

In [9]:
C_c_list_dict = {}
P_c_list_dict = {}

for n in n_array:
    c_list = c_list_dict[n]
    C_c_list = []
    P_c_list = []
    
    for c in c_list:
        C_c_list.append(calc_LSIML.calc_C_n(c))
        P_c_list.append(calc_LSIML.calc_P_n(c))
        
    C_c_list_dict[n] = C_c_list
    P_c_list_dict[n] = P_c_list

## LSIMLを計算する

### 関数を設定する

In [10]:
def calc_RV(Y):
    RV = sum(np.diff(Y) ** 2)

    return RV

In [12]:
def calc_daily_LSIML(Y, b_list, c_list_dict, C_c_list_dict, P_c_list_dict, alpha, overlap_rate):
    '''
    各日のLSIML,ジャンプ件数,ジャンプサイズを(b毎に)計算する
    '''
    # 対数価格を抽出し、NANを除去する
    n = len(Y) - 1
    Y = np.array(Y)

    daily_LSIML_list = []
    daily_num_of_detected_jump_list = []
    daily_size_jump_list = []

    # 各bに対して、LSIMLを計算する
    for j in range(len(b_list)):
        b = b_list[j]
        c = c_list_dict[n][j]
        C_c = C_c_list_dict[n][j]
        P_c = P_c_list_dict[n][j]
        # を計算する。
        ##quantileの手法でジャンプ検出を行う
        result_dict = calc_LSIML.calc_LSIML(
            Y,
            b,
            c,
            alpha,
            C_c,
            P_c,
            n,
            jump_detect=True,
            how_to_detect="quantile",
            overlap_rate=overlap_rate,
        )
        daily_LSIML_list.append(result_dict["LSIML"])
        daily_num_of_detected_jump_list.append(result_dict["num_of_detected_jump"])
        daily_size_jump_list.append(result_dict["sum_of_jump"])

    return daily_LSIML_list, daily_num_of_detected_jump_list, daily_size_jump_list

In [13]:
df = sec1
# 初期値
RV_list = []
LSIML_list = []
num_of_detected_jump_list = []
size_jump_list = []

# 日付でグループ化する
groups = df.groupby(df.date)

count = 0
for date in Days_list:
    count += 1
    print(count)
    
    # LSIMLを計算する
    Y = groups.get_group(date)["log_price"]
    daily_LSIML_list, daily_num_of_detected_jump_list, daily_size_jump_list = calc_daily_LSIML(Y, b_list, c_list_dict, C_c_list_dict, P_c_list_dict, alpha, overlap_rate = 0)
    
    LSIML_list.append(daily_LSIML_list)
    num_of_detected_jump_list.append(daily_num_of_detected_jump_list)
    size_jump_list.append(daily_size_jump_list)

    # RVを計算する
    RV_list.append(calc_RV(Y))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
LSIML_result_df = pd.DataFrame(
    np.array(LSIML_list),
    columns=[
        "LSIML_" + "b=" + str(b_list[0]) + "_sec1",
        "LSIML_" + "b=" + str(b_list[1]) + "_sec1",
        "LSIML_" + "b=" + str(b_list[2]) + "_sec1",
    ],
)

In [15]:
# プロット用に時間軸を保存
time = np.array(pd.to_datetime(Days_list))

# RV列をdfに結合
df_sec1 = pd.DataFrame([time, RV_list]).T
df_sec1.columns = ["date", "RV_sec1"]
df_sec1 = pd.merge(df_sec1, LSIML_result_df, right_index=True, left_index=True)

In [16]:
df_sec1

,date,RV_sec1,LSIML_b=10_sec1,LSIML_b=50_sec1,LSIML_b=100_sec1
0,2018-09-03,0.0001,0.000034,0.000046,0.000046
1,2018-09-04,0.000152,0.000040,0.000041,0.000047
2,2018-09-05,0.000086,0.000025,0.000031,0.000036
3,2018-09-06,0.000094,0.000038,0.000034,0.000039
4,2018-09-07,0.000123,0.000039,0.000055,0.000044
...,...,...,...,...,...
699,2021-07-26,0.000125,0.000025,0.000035,0.000046
700,2021-07-27,0.000091,0.000023,0.000024,0.000031
701,2021-07-28,0.000105,0.000031,0.000035,0.000044
702,2021-07-29,0.000089,0.000027,0.000036,0.000035
